In [1]:
import numpy as np
import cv2

# 28. 직선 검출

In [2]:
# 직선 검출(Line Detection)
# 허프 변환(Hough Transform) 이용
# 허프 변환 : 이미지에서 직선을 찾는 가장 보편적인 알고리즘
# 선과 같은 단순한 형태를 빠르게 검출할 수 있으며, 직선을 찾아 이미지나 영상을 보정하거나 복원
# 이미지 내의 어떤 점이라도 선 집합의 일부일 수 있다는 가정하에 직선의 방정식을 이용해 직선을 검출

# 표준 허프 변환(Standard Hough Transform)
# 입력 이미지(x, y 평면) 내의 점 를 지나는 직선의 방정식을 구함
# y = ax+b로 표현 가능
# -> 수평(기울기0), 수직(기울기 무한대) -> 직선을 표현하는 좋은 방식이 아님
# -> 삼각함수 이용, 각 선을 극좌표(ρ, θ)의 점으로 변환

# 멀티 스케일 허프 변환(Multi-Scale Hough Transform)
# 표준 허프 변환 개선
# 거리(ρ)와 각도(θ)의 값을 조정해 사용
# 두 값을 조정하는 방법으로 조금 더 우수한 검출 가능

In [18]:
# 원본 이미지(src)와 결과 이미지(dst)를 선언
src = cv2.imread("Image/road.jpg")
dst = src.copy()

# 전처리를 진행하기 위해 그레이스케일 이미지(gray)와 케니 엣지 이미지(canny)를 사용
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

# cv2.Canny(img, 임곗값, 임곗값, 커널크기 = 5, L2gradient = True)
canny = cv2.Canny(gray, 5000, 1500, apertureSize = 5, L2gradient = True)

# 직선 검출 진행
# cv2.HoughLines(검출 이미지, 거리, 각도, 임곗값, 거리 약수, 각도 약수, 최소 각도, 최대 각도)
# 거리와 각도 : 누산 평면에서 사용되는 해상도, 거리(0~1사이 실수), 각도(0~180라디안)
# 임곗값 : 허프 변환 알고리즘이 직선을 결정하기 위해 만족해야 하는 누산 평면의 값
# 누산 평면 : 각도 × 거리의 차원을 갖는 2차원 히스토그램으로 구성
# 거리 약수, 각도 약수 : 거리와 각도에 대한 약수(divisor)
  # 두 값 모두 0의 값을 인수로 활용할 경우 : 표준 허프 변환
  # 하나 이상의 값이 0이 아님 : 멀티 스케일 허프 변환이 적용
# 최소 각도, 최대 각도 : 검출할 각도의 범위
lines = cv2.HoughLines(canny, 0.8, np.pi / 180, 150, srn = 100, stn = 200, min_theta = 0, max_theta = np.pi)


# lines -> (N, 1, 2)차원 형태
for i in lines: 
    
    # 검출된 거리(rho)와 각도(theta)
    rho, theta = i[0][0], i[0][1]
    a, b = np.cos(theta), np.sin(theta)
    
    # circle의 좌표에 정수가 들어가야 함
    # x=rho*cosθ, y=rho*sinθ
    x0, y0 = int(a*rho), int(b*rho)

    # HoughLines는 좌표를 알려주는 함수가 아님
    # 가장 직선일 가능성이 높은 거리와 각도를 검출하는 함수
    # 그러므로 출력 이미지 위에 표현하기 위해 x0와 y0를 직선의 방정식 선분을 따라 평행이동시켜 선을 그림
    scale = src.shape[0] + src.shape[1]

    x1 = int(x0 + scale * -b)
    y1 = int(y0 + scale * a)
    x2 = int(x0 - scale * -b)
    y2 = int(y0 - scale * a)

    cv2.line(dst, (x1, y1), (x2, y2), (0, 0, 255), 2)
    cv2.circle(dst, (x0, y0), 3, (255, 0, 0), 5, cv2.FILLED)

dst=cv2.resize(dst, (0,0), fx=0.5, fy=0.5)

cv2.imshow("dst", dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# 점진성 확률적 허프 변환(Progressive Probabilistic Hough Transform)
# 또 다른 허프 변환 함수
# 앞선 알고리즘은 모든 점에 대해 직선의 방정식을 세워 계산하기 때문에 비교적 많은 시간이 소모
# 따라서 앞선 알고리즘을 최적화한 방식
# 임의의 점 일부만 누적해서 계산 -> 일부의 점만 사용하기 때문에 확률적 -> 시간 대폭 줄어듬
# 시작점과 끝점을 반환

In [2]:
src = cv2.imread("Image/road.jpg")
dst = src.copy()
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
canny = cv2.Canny(gray, 5000, 1500, apertureSize = 5, L2gradient = True)

# 직선 검출 진행
# cv2.HoughLinesP(검출 이미지, 거리, 각도, 임곗값, 최소 선 길이, 최대 선 간격)
# 최소 선 길이 : 검출된 직선이 가져야 하는 최소한의 선 길이
# 최대 선 간격 : 검출된 직선들 사이의 최대 허용 간격
lines = cv2.HoughLinesP(canny, 0.8, np.pi / 180, 90, minLineLength = 10, maxLineGap = 100)


# lines -> (N, 1, 4)차원 형태
# 마지막 차원에서 x1, y1, x2, y2의 순서로 시작점과 끝점을 표시
for i in lines:
    cv2.line(dst, (int(i[0][0]), int(i[0][1])), (int(i[0][2]), int(i[0][3])), (0, 0, 255), 2)

dst=cv2.resize(dst, (0,0), fx=0.5, fy=0.5)

cv2.imshow("dst", dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 29. 원 검출

In [ ]:
# 원 검출(Circle Detection)
# 허프 원 변환(Hough Circle Transform) 알고리즘을 활용
# 2차원이 아닌 3차원 누산 평면으로 검출
# 원의 중심점 x, 원의 중심점 y, 원의 반경 r을 활용해 누산 평면을 구성
# 누산 평면은 2차원 공간(x, y)에서 3차원 공간(a, b, r)으로 변환
# 이미지에서 가장자리를 검출
# 3차원 히스토그램에서 돗수가 높은 (a, b, r)을 선택
# 필요한 메모리가 너무 많아 비효율적(가장 긴 변의 길이가 N이라면 N^3바이트의 메모리 필요)

# 2차원 방식을 사용 -> 2단계로 나눠 계산 -> 2단계 허프 변환(Two stage Hough Transform) 방법
# 가장자리에 그레이디언트 방법을 이용해 원의 중심점(a, b)에 대한 2차원 히스토그램을 선정
# 모든 점에 대해 최소 거리에서 최대 거리까지 기울기의 선분을 따라 누산 평면의 모든 점을 증가시킴
# 중심점을 선택하기 위해 중심점 후보군에서 임곗값보다 크고 인접한 점보다 큰 점을 중심점으로 사용
# 선정된 중심점(a, b)와 가장자리의 좌표를 원의 방정식에 대입해 반지름 r의 1차원 히스토그램으로 판단
# 필요한 메모리가 줄어듦(가장 긴 변의 길이가 N이라면 N^2+N바이트의 메모리 필요)


In [16]:
src = cv2.imread("Image/colorball.jpg")
dst = src.copy()
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

# 원 검출
# cv2.HoughCircles(검출 이미지, 검출 방법, 해상도 비율, 최소 거리, 캐니 엣지 임곗값, 중심 임곗값, 최소 반지름, 최대 반지름)
# 검출 방법 : 항상 2단계 허프 변환 방법(21HT, 그레이디언트)만 사용
# 해상도 비율 : 원의 중심을 검출하는 데 사용되는 누산 평면의 해상도, 1: 입력한 이미지와 동일한 해상도, 2: 누산 평면의 해상도가 절반으로 줄어 입력 이미지의 크기와 반비례
# 최소 거리 : 일차적으로 검출된 원과 원 사이의 최소 거리
# 캐니 엣지 임곗값 : 허프 변환에서 자체적으로 캐니 엣지를 적용하게 되는데, 이때 사용되는 상위 임곗값
# 하위 임곗값은 자동으로 할당되며, 상위 임곗값의 절반에 해당하는 값을 사용

# 중심 임곗값 : 그레이디언트 방법에 적용된 중심 히스토그램(누산 평면)에 대한 임곗값, 낮을수록 많은 원
# 최소 반지름, 최대 반지름 : 검출될 원의 반지름 범위, 0: 제한조건X
# 최대 반지름에 음수를 입력할 경우 검출된 원의 중심만 반환
circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 2, 100, param1 = 250, param2 = 10, minRadius = 80, maxRadius = 120)

# circles: (1, N, 3)차원 형태
# 중심점(x, y)과 반지름(r)이 저장
for i in circles[0]:
    
    cv2.circle(dst, (int(i[0]), int(i[1])), int(i[2]), (255, 255, 255), 5)

dst=cv2.resize(dst, (0,0), fx=0.5, fy=0.5)

cv2.imshow("dst", dst)
cv2.waitKey(0)
cv2.destroyAllWindows()